# 🚀 CADDi セールスアナリティクス v3.0

## 💡 AIと対話しながらデータ分析

**日本語で話しかけるだけ**で、AIがあなたの営業データを即座に分析します。

---

### ⚡ 30秒で分析開始

1. **下のボタンをクリック** → 自動セットアップ
2. **質問を選ぶか入力** → AIが分析
3. **結果を確認** → グラフと要約を自動生成

---

In [ ]:
# 🎯 ワンクリック設定（このセルを実行するだけ！）

import warnings
warnings.filterwarnings('ignore')

print("⏳ セットアップ中...（30秒程度かかります）\n")

# 必要なパッケージを自動インストール
import subprocess
import sys

packages = [
    'google-cloud-bigquery', 'google-generativeai', 
    'pandas', 'plotly', 'ipywidgets'
]

for package in packages:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", package], 
                   capture_output=True)

# ライブラリをインポート
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import google.generativeai as genai
from google.cloud import bigquery

print("✅ パッケージの準備完了\n")

# Google Cloud認証
print("⏳ Google Cloud接続中...\n")
try:
    from google.colab import auth
    auth.authenticate_user()
    print("✅ Google Cloud接続成功\n")
except:
    print("ℹ️ ローカル環境で実行中\n")

# Gemini API設定
print("⏳ AI設定中...\n")
try:
    from google.colab import userdata
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("✅ AI設定完了\n")
except:
    print("⚠️ APIキーの設定が必要です")
    print("📝 設定方法:")
    print("1. 左側の🔑アイコンをクリック")
    print("2. 'シークレットを追加'")
    print("3. 名前: GEMINI_API_KEY")
    print("4. 値: 提供されたAPIキー\n")

# BigQueryクライアント初期化
PROJECT_ID = 'esperanto-drawer-prod'
client = bigquery.Client(project=PROJECT_ID)
model = genai.GenerativeModel('gemini-1.5-flash')

print("\n🎉 セットアップ完了！\n")
print("=" * 60)
print("\n📊 データベース構造を詳しく探索してみましょう...\n")

# データ探索ダッシュボードを作成
def create_data_explorer():
    \"\"\"インタラクティブなデータ探索ダッシュボードを作成\"\"\"
    
    try:
        # テーブルの完全なスキーマ情報を取得
        def get_full_schema(table_name):
            schema_query = f\"\"\"
            SELECT 
                column_name,
                data_type,
                is_nullable,
                CASE 
                    WHEN column_name LIKE '%date%' OR column_name LIKE '%time%' THEN 'temporal'
                    WHEN column_name LIKE '%amount%' OR column_name LIKE '%revenue%' THEN 'financial'
                    WHEN column_name LIKE '%name%' OR column_name LIKE '%id%' THEN 'identifier'
                    WHEN column_name LIKE '%flag%' OR column_name LIKE '%status%' THEN 'categorical'
                    ELSE 'other'
                END as category
            FROM `{PROJECT_ID}.dm_business_planning.INFORMATION_SCHEMA.COLUMNS`
            WHERE table_name = '{table_name}'
            ORDER BY ordinal_position
            \"\"\"
            return client.query(schema_query).to_dataframe()
        
        # サンプルデータとユニーク値を取得
        def get_sample_and_stats(table_name, limit=1000):
            sample_query = f\"\"\"
            SELECT * FROM `{PROJECT_ID}.dm_business_planning.{table_name}`
            LIMIT {limit}
            \"\"\"
            return client.query(sample_query).to_dataframe()
        
        # 各テーブルの情報を取得
        print("⏳ テーブル情報を取得中...\n")
        
        account_schema = get_full_schema('salesforce_account_mart')
        opportunity_schema = get_full_schema('salesforce_opportunity_mart')
        
        account_data = get_sample_and_stats('salesforce_account_mart', 100)
        opportunity_data = get_sample_and_stats('salesforce_opportunity_mart', 100)
        
        # カラム説明辞書（Salesforce標準フィールド）
        column_descriptions = {
            'account_id': '顧客の一意識別子',
            'account_name': '顧客企業名',
            'company_size': '企業規模 (ENT=大企業, MID=中企業, SB=小企業)',
            'industry': '業界分類',
            'annual_revenue': '年間売上高',
            'number_of_employees': '従業員数',
            'won_flag': '受注実績フラグ（True/False）',
            'opportunity_id': '商談の一意識別子',
            'opportunity_name': '商談名',
            'stage_name': '商談ステージ（Prospecting, Qualification, Closed Wonなど）',
            'amount': '商談金額',
            'close_date': 'クローズ予定日',
            'probability': '成約確度（%）',
            'created_date': '作成日',
            'owner_id': '営業担当者ID',
        }
        
        def create_table_explorer(table_name, schema_df, data_df, title):
            \"\"\"個別テーブルの探索パネルを作成\"\"\"
            
            # テーブル統計
            stats_html = f\"\"\"
            <div style=\"display: flex; gap: 15px; margin-bottom: 20px;\">
                <div style=\"background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                            color: white; padding: 15px; border-radius: 10px; flex: 1; text-align: center;\">
                    <div style=\"font-size: 24px; font-weight: bold;\">{len(data_df)}</div>
                    <div>サンプル行数</div>
                </div>
                <div style=\"background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
                            color: white; padding: 15px; border-radius: 10px; flex: 1; text-align: center;\">
                    <div style=\"font-size: 24px; font-weight: bold;\">{len(schema_df)}</div>
                    <div>総カラム数</div>
                </div>
                <div style=\"background: linear-gradient(135deg, #4facfe 0%, #00f2fe 100%); 
                            color: white; padding: 15px; border-radius: 10px; flex: 1; text-align: center;\">
                    <div style=\"font-size: 24px; font-weight: bold;\">{len([c for c in data_df.columns if data_df[c].dtype in ['int64', 'float64']])}</div>
                    <div>数値カラム</div>
                </div>
            </div>
            \"\"\"
            
            display(HTML(f\"<h3>{title}</h3>\"))
            display(HTML(stats_html))
            
            # カラム辞書作成
            column_info = []
            for _, row in schema_df.iterrows():
                col_name = row['column_name']
                description = column_descriptions.get(col_name, '🤔 このカラムの詳細説明が必要な場合はお聞かせください')
                
                # サンプル値とユニーク値数を取得
                if col_name in data_df.columns:
                    unique_count = data_df[col_name].nunique()
                    sample_values = data_df[col_name].dropna().head(3).tolist()
                    null_count = data_df[col_name].isna().sum()
                else:
                    unique_count = 0
                    sample_values = []
                    null_count = 0
                
                column_info.append({
                    'カラム名': col_name,
                    '説明': description,
                    'データ型': row['data_type'],
                    'ユニーク値数': unique_count,
                    'サンプル値': str(sample_values)[:50] + '...' if len(str(sample_values)) > 50 else str(sample_values),
                    '欠損値数': null_count
                })
            
            column_df = pd.DataFrame(column_info)
            
            # スクロール可能なテーブル表示
            display(HTML(\"<h4>📋 カラム詳細情報</h4>\"))
            display(column_df.style.set_properties(**{
                'background-color': '#f8f9fa',
                'border': '1px solid #dee2e6',
                'padding': '8px',
                'font-size': '12px'
            }).set_table_styles([
                {'selector': 'th', 'props': [('background-color', '#e9ecef'), ('font-weight', 'bold')]},
                {'selector': 'tr:hover', 'props': [('background-color', '#f5f5f5')]}
            ]))
            
            # 実際のデータサンプル
            display(HTML(\"<h4>📊 データサンプル（最新20行）</h4>\"))
            display(data_df.head(20).style.set_properties(**{
                'background-color': '#ffffff',
                'border': '1px solid #dee2e6',
                'padding': '6px',
                'font-size': '11px'
            }).set_table_styles([
                {'selector': 'th', 'props': [('background-color', '#007bff'), ('color', 'white')]},
                {'selector': 'tr:nth-child(even)', 'props': [('background-color', '#f8f9fa')]}
            ]))
            
            # 分析提案
            suggestions = []
            if 'amount' in data_df.columns:
                suggestions.append(f\"💰 '{table_name}の金額上位10件を見せて'\")\n            if 'stage_name' in data_df.columns:
                suggestions.append(f\"📊 '{table_name}をステージ別に集計して'\")\n            if 'company_size' in data_df.columns:
                suggestions.append(f\"🏢 'company_size別の分布を教えて'\")\n            if 'close_date' in data_df.columns:
                suggestions.append(f\"📅 '月別の{table_name}推移を見せて'\")\n            \n            if suggestions:\n                suggestions_html = \"<br>\".join([f\"<li style='margin: 5px 0;'>{s}</li>\" for s in suggestions])\n                display(HTML(f\"\"\"\n                <div style=\"background-color: #e8f5e9; padding: 15px; border-radius: 10px; margin-top: 20px;\">\n                    <h4>💡 このテーブルで試せる分析例：</h4>\n                    <ul style=\"margin: 10px 0; padding-left: 20px;\">\n                        {suggestions_html}\n                    </ul>\n                </div>\n                \"\"\"))\n        \n        # タブ形式でテーブル情報を表示\n        tab1_output = widgets.Output()\n        tab2_output = widgets.Output()\n        tab3_output = widgets.Output()\n        \n        with tab1_output:\n            create_table_explorer(\n                'salesforce_account_mart',\n                account_schema, \n                account_data,\n                '🏢 顧客データ (Account Mart)'\n            )\n        \n        with tab2_output:\n            create_table_explorer(\n                'salesforce_opportunity_mart',\n                opportunity_schema,\n                opportunity_data, \n                '💼 商談データ (Opportunity Mart)'\n            )\n        \n        with tab3_output:\n            # 全体統計とリレーション説明\n            display(HTML(\"\"\"\n            <div style=\"background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); \n                        padding: 20px; border-radius: 15px; margin-bottom: 20px;\">\n                <h3>🔗 テーブル間の関係</h3>\n                <p><strong>顧客データ ↔ 商談データ</strong></p>\n                <p>📌 <code>account_id</code> で紐づいています</p>\n                <p>💡 <strong>結合分析例:</strong> \"ENT顧客の商談成約率を計算して\" → 顧客テーブルで企業規模を確認し、商談テーブルで成約状況を分析</p>\n            </div>\n            \n            <div style=\"background-color: #fff3cd; padding: 15px; border-radius: 10px;\">\n                <h4>❓ カラムの意味がわからない場合</h4>\n                <p>上記の説明で不明なカラムがある場合は、遠慮なく <strong>「○○カラムの意味を教えて」</strong> とお聞きください！</p>\n                <p>推測ではなく、正確な情報をお答えします。</p>\n            </div>\n            \"\"\"))\n            \n            # よくある分析パターンを表示\n            common_analyses = [\n                {\n                    'title': '📈 売上・収益分析',\n                    'examples': [\n                        '今月の売上トップ10社を金額とともに教えて',\n                        '前年同月比での売上成長率を計算して',\n                        '月別の売上推移をグラフで見せて'\n                    ]\n                },\n                {\n                    'title': '🎯 商談・営業分析', \n                    'examples': [\n                        '現在進行中の商談をステージ別に件数と金額で集計',\n                        '営業担当者別の成約率ランキングを見せて',\n                        '平均商談期間をステージ別に教えて'\n                    ]\n                },\n                {\n                    'title': '🏢 顧客セグメント分析',\n                    'examples': [\n                        'company_size別の顧客数と平均売上を教えて',\n                        '業界別の顧客分布と売上を見せて', \n                        'ENT顧客の新規獲得数を月別で追って'\n                    ]\n                }\n            ]\n            \n            for analysis in common_analyses:\n                examples_html = \"<br>\".join([f\"<li style='margin: 8px 0; padding: 5px; background-color: white; border-radius: 5px;'>{ex}</li>\" for ex in analysis['examples']])\n                display(HTML(f\"\"\"\n                <div style=\"background-color: #f8f9fa; padding: 15px; border-radius: 10px; margin: 15px 0;\">\n                    <h4>{analysis['title']}</h4>\n                    <ul style=\"list-style: none; padding: 0;\">\n                        {examples_html}\n                    </ul>\n                </div>\n                \"\"\"))\n        \n        # タブウィジェット作成\n        tabs = widgets.Tab(children=[tab1_output, tab2_output, tab3_output])\n        tabs.set_title(0, '🏢 顧客データ')\n        tabs.set_title(1, '💼 商談データ') \n        tabs.set_title(2, '📚 分析ガイド')\n        \n        display(HTML(\"\"\"\n        <h2 style=\"background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); \n                   color: white; padding: 15px; border-radius: 10px; text-align: center;\">\n            📊 データ構造エクスプローラー\n        </h2>\n        <p style=\"text-align: center; color: #666; margin-bottom: 20px;\">\n            各タブをクリックして、利用可能なデータを詳しく確認してください\n        </p>\n        \"\"\"))\n        \n        display(tabs)\n        \n    except Exception as e:\n        display(HTML(f\"\"\"\n        <div style=\"background-color: #ffebee; padding: 15px; border-radius: 10px;\">\n            <h4>⚠️ データ取得エラー</h4>\n            <p>データベースへの接続に問題があります: {str(e)[:200]}</p>\n            <p>分析機能は利用可能ですが、事前のデータ確認ができません。</p>\n        </div>\n        \"\"\"))\n\n# データエクスプローラーを表示\ncreate_data_explorer()\n\nprint(\"\\n\" + \"=\" * 60)\nprint(\"\\n✅ 準備完了！下のセルで分析を開始してください\")"

# 🎮 インタラクティブ分析インターフェース

# グローバル変数
analysis_results = []

# クイック分析ボタンの定義
quick_analyses = {
    "📈 今月の売上TOP10": "今月の売上が高い顧客トップ10を金額とともに教えて",
    "📊 商談ステータス": "現在進行中の商談をステージ別に件数と金額で集計して",
    "🎯 成約率ランキング": "営業担当者別の商談成約率を高い順に教えて",
    "💰 大口案件リスト": "商談金額1000万円以上の案件を金額順に見せて",
    "📅 月次売上推移": "2024年の月別売上推移をグラフで見せて",
    "🏢 企業セグメント": "company_size別の顧客数と売上合計を教えて",
    "🆕 新規顧客分析": "今月獲得した新規顧客の一覧と売上を見せて",
    "⏰ リードタイム": "商談の平均リードタイムをステージ別に教えて"
}

# スタイル定義
style = """
<style>
.analysis-container {
    background-color: #f8f9fa;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.result-box {
    background-color: white;
    padding: 15px;
    margin-top: 10px;
    border-radius: 8px;
    border-left: 4px solid #007bff;
}
.summary-text {
    color: #333;
    font-size: 14px;
    line-height: 1.6;
}
.metric {
    display: inline-block;
    padding: 10px 15px;
    margin: 5px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 20px;
    font-weight: bold;
}
.save-button {
    display: inline-block;
    margin: 5px;
    padding: 8px 15px;
    background-color: #28a745;
    color: white;
    border: none;
    border-radius: 5px;
    cursor: pointer;
    font-size: 12px;
}
.save-button:hover {
    background-color: #218838;
}
</style>
"""

# 分析結果の保存機能（初心者向け）
def create_save_options(df, question, summary):
    """初心者にとって最もわかりやすい保存オプションを提供"""
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    safe_question = question.replace('/', '_').replace('?', '').replace(':', '')[:30]
    
    # 保存オプションの説明
    save_options_html = f"""
    <div style="background-color: #e8f5e9; padding: 15px; border-radius: 10px; margin-top: 15px;">
        <h4>💾 分析結果を保存しますか？</h4>
        <p style="color: #666; font-size: 14px;">用途に合わせて保存形式を選んでください：</p>
    </div>
    """
    
    display(HTML(save_options_html))
    
    # 保存ボタンの作成
    save_buttons = []
    
    # 1. Excel保存（最も一般的）
    excel_button = widgets.Button(
        description='📊 Excel形式で保存',
        button_style='success',
        tooltip='データをExcelファイル(.xlsx)として保存。Excelで開いて編集可能',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 2. CSV保存（互換性重視）
    csv_button = widgets.Button(
        description='📋 CSV形式で保存',
        button_style='primary',
        tooltip='データをCSVファイルとして保存。どのソフトでも開ける',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 3. レポート保存（要約付き）
    report_button = widgets.Button(
        description='📝 レポート形式で保存',
        button_style='info',
        tooltip='データと要約をまとめたHTMLレポートを作成',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # 4. 画像保存（グラフ）
    image_button = widgets.Button(
        description='📷 グラフを画像保存',
        button_style='warning',
        tooltip='グラフを画像ファイル(PNG)として保存',
        layout=widgets.Layout(width='200px', height='40px')
    )
    
    # メッセージ表示用
    message_output = widgets.Output()
    
    def save_as_excel(b):
        try:
            filename = f'分析結果_{safe_question}_{timestamp}.xlsx'
            df.to_excel(filename, index=False, engine='openpyxl')
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: green;">✅ Excelファイルに保存しました: {filename}</div>'))
        except Exception as e:
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: red;">❌ 保存エラー: {str(e)}</div>'))
    
    def save_as_csv(b):
        try:
            filename = f'分析結果_{safe_question}_{timestamp}.csv'
            df.to_csv(filename, index=False, encoding='utf-8-sig')
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: green;">✅ CSVファイルに保存しました: {filename}</div>'))
        except Exception as e:
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: red;">❌ 保存エラー: {str(e)}</div>'))
    
    def save_as_report(b):
        try:
            filename = f'分析レポート_{safe_question}_{timestamp}.html'
            
            # HTMLレポート作成
            report_html = f"""
            <!DOCTYPE html>
            <html>
            <head>
                <meta charset="utf-8">
                <title>分析レポート - {question}</title>
                <style>
                    body {{ font-family: Arial, sans-serif; margin: 20px; }}
                    .header {{ background-color: #007bff; color: white; padding: 20px; border-radius: 10px; }}
                    .summary {{ background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin: 20px 0; }}
                    table {{ border-collapse: collapse; width: 100%; margin-top: 20px; }}
                    th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
                    th {{ background-color: #f2f2f2; }}
                </style>
            </head>
            <body>
                <div class="header">
                    <h1>📊 CADDi 分析レポート</h1>
                    <p>作成日時: {datetime.now().strftime('%Y年%m月%d日 %H時%M分')}</p>
                </div>
                
                <h2>🔍 分析内容</h2>
                <p><strong>質問:</strong> {question}</p>
                <p><strong>データ件数:</strong> {len(df)}件</p>
                
                <div class="summary">
                    <h3>💡 分析結果のポイント</h3>
                    <p>{summary.replace(chr(10), '<br>')}</p>
                </div>
                
                <h3>📋 詳細データ</h3>
                {df.to_html(index=False, escape=False, table_id='data-table')}
            </body>
            </html>
            """
            
            with open(filename, 'w', encoding='utf-8') as f:
                f.write(report_html)
                
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: green;">✅ HTMLレポートに保存しました: {filename}<br>ブラウザで開くと見やすいレポートが表示されます</div>'))
        except Exception as e:
            with message_output:
                clear_output(wait=True)
                display(HTML(f'<div style="color: red;">❌ 保存エラー: {str(e)}</div>'))
    
    def save_as_image(b):
        with message_output:
            clear_output(wait=True)
            display(HTML('<div style="color: orange;">📷 グラフ画像の保存は、グラフ上で右クリック →「画像として保存」を選択してください</div>'))
    
    # イベントハンドラー登録
    excel_button.on_click(save_as_excel)
    csv_button.on_click(save_as_csv)
    report_button.on_click(save_as_report)
    image_button.on_click(save_as_image)
    
    # ボタンとメッセージ表示
    button_box = widgets.HBox([excel_button, csv_button, report_button, image_button])
    display(button_box)
    display(message_output)

# 分析実行関数
def run_analysis(question):
    """分析を実行してインタラクティブに結果を表示"""
    
    with output:
        clear_output(wait=True)
        display(HTML(style))
        
        # プログレスバー表示
        progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='分析中:',
            bar_style='info',
            style={'bar_color': '#007bff'},
            layout=widgets.Layout(width='100%', height='30px')
        )
        display(progress)
        
        try:
            # SQL生成
            progress.value = 20
            progress.description = 'SQL生成中:'
            
            prompt = f"""
            以下の質問に答えるBigQuery SQLを生成してください：
            
            質問: {question}
            
            テーブル:
            - esperanto-drawer-prod.dm_business_planning.salesforce_account_mart
              (account_id, account_name, company_size, won_flag等)
            - esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart
              (opportunity_id, account_id, amount, stage_name, close_date等)
            
            必ずLIMIT 100を付けてください。
            SQLのみを```sql```で囲んで返してください。
            """
            
            response = model.generate_content(prompt)
            sql = response.text
            
            if '```sql' in sql:
                sql = sql.split('```sql')[1].split('```')[0].strip()
            elif '```' in sql:
                sql = sql.split('```')[1].split('```')[0].strip()
            
            # データ取得
            progress.value = 50
            progress.description = 'データ取得中:'
            
            df = client.query(sql).to_dataframe()
            
            # 結果表示準備
            progress.value = 80
            progress.description = '可視化中:'
            
            clear_output(wait=True)
            display(HTML(style))
            
            # 結果サマリー
            display(HTML(f"""
            <div class="analysis-container">
                <h3>🔍 質問: {question}</h3>
                <div class="metric">{len(df)} 件のデータ</div>
                <div class="metric">{len(df.columns)} カラム</div>
            </div>
            """))
            
            # データテーブル表示
            if len(df) > 0:
                display(HTML("<h4>📊 データ（上位10件）</h4>"))
                display(df.head(10).style.set_properties(**{
                    'background-color': '#f0f2f5',
                    'border': '1px solid #ddd',
                    'padding': '8px'
                }))
                
                # グラフ生成
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
                
                if len(numeric_cols) > 0 and len(non_numeric_cols) > 0:
                    # インタラクティブグラフ
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=('棒グラフ', '円グラフ'),
                        specs=[['bar', 'pie']]
                    )
                    
                    # 棒グラフ
                    fig.add_trace(
                        go.Bar(
                            x=df[non_numeric_cols[0]].head(10),
                            y=df[numeric_cols[0]].head(10),
                            marker_color='lightblue',
                            text=df[numeric_cols[0]].head(10),
                            textposition='auto'
                        ),
                        row=1, col=1
                    )
                    
                    # 円グラフ
                    fig.add_trace(
                        go.Pie(
                            labels=df[non_numeric_cols[0]].head(5),
                            values=df[numeric_cols[0]].head(5),
                            hole=0.3
                        ),
                        row=1, col=2
                    )
                    
                    fig.update_layout(
                        height=400,
                        showlegend=False,
                        title_text=f"分析結果: {question}",
                        template='plotly_white'
                    )
                    
                    fig.show()
                
                # AI要約
                progress.value = 90
                progress.description = '要約生成中:'
                
                summary_prompt = f"""
                データ分析結果を営業チーム向けに要約してください：
                
                質問: {question}
                データ: {len(df)}件
                
                重要なポイントを3つ箇条書きで。
                """
                
                summary = model.generate_content(summary_prompt).text
                
                display(HTML(f"""
                <div class="result-box">
                    <h4>💡 分析結果のポイント</h4>
                    <div class="summary-text">{summary.replace(chr(10), '<br>')}</div>
                </div>
                """))
                
                # 保存オプション表示
                create_save_options(df, question, summary)
                
                # 結果を履歴に保存
                analysis_results.append({
                    'timestamp': datetime.now(),
                    'question': question,
                    'data': df,
                    'summary': summary
                })
            
            progress.value = 100
            progress.description = '完了!'
            
        except Exception as e:
            clear_output(wait=True)
            display(HTML(f"""
            <div style="background-color: #ffebee; padding: 15px; border-radius: 8px; border-left: 4px solid #f44336;">
                <h4>❌ エラーが発生しました</h4>
                <p>{str(e)[:200]}</p>
                <p><strong>ヒント:</strong></p>
                <ul>
                    <li>質問をより具体的にしてみてください</li>
                    <li>期間を指定してみてください（例：2024年1月）</li>
                    <li>データ量を制限してみてください（例：上位10件）</li>
                </ul>
            </div>
            """))

# UI要素の作成
display(HTML("""
<h3 style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
           color: white; padding: 15px; border-radius: 10px; text-align: center;">
    🎯 分析したい内容を選択または入力してください
</h3>
"""))

# クイック分析ボタン
button_box = widgets.GridBox(
    children=[
        widgets.Button(
            description=label,
            button_style='primary',
            layout=widgets.Layout(width='auto', height='40px'),
            style={'button_color': '#007bff'}
        ) for label in quick_analyses.keys()
    ],
    layout=widgets.Layout(
        grid_template_columns='repeat(4, 1fr)',
        grid_gap='10px',
        width='100%'
    )
)

# カスタム質問入力
custom_input = widgets.Textarea(
    value='',
    placeholder='例：2024年1月の売上が100万円以上の顧客リストを見せて',
    description='自由入力:',
    layout=widgets.Layout(width='100%', height='60px')
)

# 実行ボタン
execute_button = widgets.Button(
    description='🚀 分析実行',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

# 出力エリア
output = widgets.Output()

# イベントハンドラ
def on_quick_button_click(b):
    question = quick_analyses[b.description]
    run_analysis(question)

def on_execute_click(b):
    if custom_input.value:
        run_analysis(custom_input.value)
    else:
        with output:
            clear_output(wait=True)
            display(HTML("""
            <div style="background-color: #fff3cd; padding: 10px; border-radius: 5px;">
                ⚠️ 質問を入力するか、上のボタンから選択してください
            </div>
            """))

# イベント登録
for button in button_box.children:
    button.on_click(on_quick_button_click)

execute_button.on_click(on_execute_click)

# UI表示
display(widgets.VBox([
    widgets.HTML("<h4>📌 クイック分析（ワンクリック）</h4>"),
    button_box,
    widgets.HTML("<br><h4>✏️ カスタム分析（自由入力）</h4>"),
    custom_input,
    execute_button,
    widgets.HTML("<br>"),
    output
]))

In [ ]:
# 🎮 インタラクティブ分析インターフェース

# グローバル変数
analysis_results = []

# クイック分析ボタンの定義
quick_analyses = {
    "📈 今月の売上TOP10": "今月の売上が高い顧客トップ10を金額とともに教えて",
    "📊 商談ステータス": "現在進行中の商談をステージ別に件数と金額で集計して",
    "🎯 成約率ランキング": "営業担当者別の商談成約率を高い順に教えて",
    "💰 大口案件リスト": "商談金額1000万円以上の案件を金額順に見せて",
    "📅 月次売上推移": "2024年の月別売上推移をグラフで見せて",
    "🏢 企業セグメント": "company_size別の顧客数と売上合計を教えて",
    "🆕 新規顧客分析": "今月獲得した新規顧客の一覧と売上を見せて",
    "⏰ リードタイム": "商談の平均リードタイムをステージ別に教えて"
}

# スタイル定義
style = """
<style>
.analysis-container {
    background-color: #f8f9fa;
    padding: 20px;
    border-radius: 10px;
    box-shadow: 0 2px 4px rgba(0,0,0,0.1);
}
.result-box {
    background-color: white;
    padding: 15px;
    margin-top: 10px;
    border-radius: 8px;
    border-left: 4px solid #007bff;
}
.summary-text {
    color: #333;
    font-size: 14px;
    line-height: 1.6;
}
.metric {
    display: inline-block;
    padding: 10px 15px;
    margin: 5px;
    background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
    color: white;
    border-radius: 20px;
    font-weight: bold;
}
</style>
"""

# 分析実行関数
def run_analysis(question):
    """分析を実行してインタラクティブに結果を表示"""
    
    with output:
        clear_output(wait=True)
        display(HTML(style))
        
        # プログレスバー表示
        progress = widgets.IntProgress(
            value=0, min=0, max=100,
            description='分析中:',
            bar_style='info',
            style={'bar_color': '#007bff'},
            layout=widgets.Layout(width='100%', height='30px')
        )
        display(progress)
        
        try:
            # SQL生成
            progress.value = 20
            progress.description = 'SQL生成中:'
            
            prompt = f"""
            以下の質問に答えるBigQuery SQLを生成してください：
            
            質問: {question}
            
            テーブル:
            - esperanto-drawer-prod.dm_business_planning.salesforce_account_mart
              (account_id, account_name, company_size, won_flag等)
            - esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart
              (opportunity_id, account_id, amount, stage_name, close_date等)
            
            必ずLIMIT 100を付けてください。
            SQLのみを```sql```で囲んで返してください。
            """
            
            response = model.generate_content(prompt)
            sql = response.text
            
            if '```sql' in sql:
                sql = sql.split('```sql')[1].split('```')[0].strip()
            elif '```' in sql:
                sql = sql.split('```')[1].split('```')[0].strip()
            
            # データ取得
            progress.value = 50
            progress.description = 'データ取得中:'
            
            df = client.query(sql).to_dataframe()
            
            # 結果表示準備
            progress.value = 80
            progress.description = '可視化中:'
            
            clear_output(wait=True)
            display(HTML(style))
            
            # 結果サマリー
            display(HTML(f"""
            <div class="analysis-container">
                <h3>🔍 質問: {question}</h3>
                <div class="metric">{len(df)} 件のデータ</div>
                <div class="metric">{len(df.columns)} カラム</div>
            </div>
            """))
            
            # データテーブル表示
            if len(df) > 0:
                display(HTML("<h4>📊 データ（上位10件）</h4>"))
                display(df.head(10).style.set_properties(**{
                    'background-color': '#f0f2f5',
                    'border': '1px solid #ddd',
                    'padding': '8px'
                }))
                
                # グラフ生成
                numeric_cols = df.select_dtypes(include=[np.number]).columns
                non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
                
                if len(numeric_cols) > 0 and len(non_numeric_cols) > 0:
                    # インタラクティブグラフ
                    fig = make_subplots(
                        rows=1, cols=2,
                        subplot_titles=('棒グラフ', '円グラフ'),
                        specs=[[{'type': 'bar'}, {'type': 'pie'}]]
                    )
                    
                    # 棒グラフ
                    fig.add_trace(
                        go.Bar(
                            x=df[non_numeric_cols[0]].head(10),
                            y=df[numeric_cols[0]].head(10),
                            marker_color='lightblue',
                            text=df[numeric_cols[0]].head(10),
                            textposition='auto'
                        ),
                        row=1, col=1
                    )
                    
                    # 円グラフ
                    fig.add_trace(
                        go.Pie(
                            labels=df[non_numeric_cols[0]].head(5),
                            values=df[numeric_cols[0]].head(5),
                            hole=0.3
                        ),
                        row=1, col=2
                    )
                    
                    fig.update_layout(
                        height=400,
                        showlegend=False,
                        title_text=f"分析結果: {question}",
                        template='plotly_white'
                    )
                    
                    fig.show()
                
                # AI要約
                progress.value = 90
                progress.description = '要約生成中:'
                
                summary_prompt = f"""
                データ分析結果を営業チーム向けに要約してください：
                
                質問: {question}
                データ: {len(df)}件
                
                重要なポイントを3つ箇条書きで。
                """
                
                summary = model.generate_content(summary_prompt).text
                
                display(HTML(f"""
                <div class="result-box">
                    <h4>💡 分析結果のポイント</h4>
                    <div class="summary-text">{summary.replace(chr(10), '<br>')}</div>
                </div>
                """))
                
                # 結果を保存
                analysis_results.append({
                    'timestamp': datetime.now(),
                    'question': question,
                    'data': df,
                    'summary': summary
                })
            
            progress.value = 100
            progress.description = '完了!'
            
        except Exception as e:
            clear_output(wait=True)
            display(HTML(f"""
            <div style="background-color: #ffebee; padding: 15px; border-radius: 8px; border-left: 4px solid #f44336;">
                <h4>❌ エラーが発生しました</h4>
                <p>{str(e)[:200]}</p>
                <p><strong>ヒント:</strong></p>
                <ul>
                    <li>質問をより具体的にしてみてください</li>
                    <li>期間を指定してみてください（例：2024年1月）</li>
                    <li>データ量を制限してみてください（例：上位10件）</li>
                </ul>
            </div>
            """))

# UI要素の作成
display(HTML("""
<h3 style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
           color: white; padding: 15px; border-radius: 10px; text-align: center;">
    🎯 分析したい内容を選択または入力してください
</h3>
"""))

# クイック分析ボタン
button_box = widgets.GridBox(
    children=[
        widgets.Button(
            description=label,
            button_style='primary',
            layout=widgets.Layout(width='auto', height='40px'),
            style={'button_color': '#007bff'}
        ) for label in quick_analyses.keys()
    ],
    layout=widgets.Layout(
        grid_template_columns='repeat(4, 1fr)',
        grid_gap='10px',
        width='100%'
    )
)

# カスタム質問入力
custom_input = widgets.Textarea(
    value='',
    placeholder='例：2024年1月の売上が100万円以上の顧客リストを見せて',
    description='自由入力:',
    layout=widgets.Layout(width='100%', height='60px')
)

# 実行ボタン
execute_button = widgets.Button(
    description='🚀 分析実行',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

# 出力エリア
output = widgets.Output()

# イベントハンドラ
def on_quick_button_click(b):
    question = quick_analyses[b.description]
    run_analysis(question)

def on_execute_click(b):
    if custom_input.value:
        run_analysis(custom_input.value)
    else:
        with output:
            clear_output(wait=True)
            display(HTML("""
            <div style="background-color: #fff3cd; padding: 10px; border-radius: 5px;">
                ⚠️ 質問を入力するか、上のボタンから選択してください
            </div>
            """))

# イベント登録
for button in button_box.children:
    button.on_click(on_quick_button_click)

execute_button.on_click(on_execute_click)

# UI表示
display(widgets.VBox([
    widgets.HTML("<h4>📌 クイック分析（ワンクリック）</h4>"),
    button_box,
    widgets.HTML("<br><h4>✏️ カスタム分析（自由入力）</h4>"),
    custom_input,
    execute_button,
    widgets.HTML("<br>"),
    output
]))

---

## 📊 分析履歴ダッシュボード

**これまでの分析結果を一覧で確認**

In [ ]:
# 📊 分析履歴をインタラクティブに表示

def show_history_dashboard():
    """分析履歴をダッシュボード形式で表示"""
    
    if not analysis_results:
        display(HTML("""
        <div style="background-color: #e3f2fd; padding: 20px; border-radius: 10px; text-align: center;">
            <h3>📭 まだ分析履歴がありません</h3>
            <p>上のセルで分析を実行すると、ここに履歴が表示されます</p>
        </div>
        """))
        return
    
    # タブ作成
    tab_contents = []
    tab_titles = []
    
    for i, result in enumerate(analysis_results[-5:], 1):  # 最新5件
        # タブタイトル
        time_str = result['timestamp'].strftime('%H:%M')
        tab_titles.append(f"分析{i} ({time_str})")
        
        # タブ内容
        df = result['data']
        
        # メトリクスカード
        metrics_html = f"""
        <div style="display: flex; gap: 10px; margin-bottom: 15px;">
            <div style="flex: 1; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        color: white; padding: 15px; border-radius: 10px; text-align: center;">
                <div style="font-size: 24px; font-weight: bold;">{len(df)}</div>
                <div>データ件数</div>
            </div>
            <div style="flex: 1; background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%); 
                        color: white; padding: 15px; border-radius: 10px; text-align: center;">
                <div style="font-size: 24px; font-weight: bold;">{len(df.columns)}</div>
                <div>カラム数</div>
            </div>
        </div>
        """
        
        # 内容HTML
        content_html = f"""
        <div style="padding: 10px;">
            <h4>🔍 質問: {result['question']}</h4>
            {metrics_html}
            <div style="background-color: #f8f9fa; padding: 15px; border-radius: 8px; margin-top: 10px;">
                <h5>💡 要約</h5>
                <p>{result['summary']}</p>
            </div>
        </div>
        """
        
        tab_contents.append(widgets.HTML(content_html))
    
    # タブウィジェット作成
    tabs = widgets.Tab(children=tab_contents)
    for i, title in enumerate(tab_titles):
        tabs.set_title(i, title)
    
    # ダッシュボード表示
    display(HTML("""
    <h3 style="background: linear-gradient(135deg, #fa709a 0%, #fee140 100%); 
               color: white; padding: 15px; border-radius: 10px; text-align: center;">
        📈 分析履歴ダッシュボード
    </h3>
    """))
    
    display(tabs)
    
    # CSVエクスポートボタン
    export_button = widgets.Button(
        description='💾 履歴をCSV保存',
        button_style='info',
        layout=widgets.Layout(width='200px')
    )
    
    def export_history(b):
        history_df = pd.DataFrame([
            {
                '時刻': r['timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
                '質問': r['question'],
                'データ件数': len(r['data']),
                '要約': r['summary'][:100]
            }
            for r in analysis_results
        ])
        
        filename = f"analysis_history_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        history_df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        display(HTML(f"""
        <div style="background-color: #d4edda; padding: 10px; border-radius: 5px; margin-top: 10px;">
            ✅ 履歴を {filename} に保存しました
        </div>
        """))
    
    export_button.on_click(export_history)
    display(export_button)

# ダッシュボード表示
show_history_dashboard()

---

## 🎯 営業KPIモニター

**重要指標をリアルタイムで監視**

In [ ]:
# 📊 KPIダッシュボード（自動更新機能付き）

def create_kpi_dashboard():
    """営業KPIをリアルタイム表示"""
    
    display(HTML("""
    <h3 style="background: linear-gradient(135deg, #a8edea 0%, #fed6e3 100%); 
               padding: 15px; border-radius: 10px; text-align: center;">
        🎯 営業KPIモニター
    </h3>
    """))
    
    # KPI定義
    kpis = [
        ("今月の売上合計", "SELECT SUM(amount) as value FROM `esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart` WHERE DATE(close_date) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)"),
        ("今月の新規商談数", "SELECT COUNT(*) as value FROM `esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart` WHERE DATE(created_date) >= DATE_SUB(CURRENT_DATE(), INTERVAL 30 DAY)"),
        ("平均商談金額", "SELECT AVG(amount) as value FROM `esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart` WHERE amount > 0"),
        ("成約率", "SELECT COUNT(CASE WHEN stage_name = 'Closed Won' THEN 1 END) * 100.0 / COUNT(*) as value FROM `esperanto-drawer-prod.dm_business_planning.salesforce_opportunity_mart`")
    ]
    
    # プログレスバー
    progress = widgets.IntProgress(
        value=0, min=0, max=len(kpis),
        description='読込中:',
        bar_style='info'
    )
    display(progress)
    
    kpi_values = []
    
    for i, (name, query) in enumerate(kpis):
        try:
            result = client.query(query).to_dataframe()
            value = result['value'].iloc[0] if not result.empty else 0
            
            # 値のフォーマット
            if '金額' in name or '売上' in name:
                formatted_value = f"¥{value:,.0f}"
            elif '率' in name:
                formatted_value = f"{value:.1f}%"
            else:
                formatted_value = f"{value:,.0f}"
            
            kpi_values.append((name, formatted_value))
        except:
            kpi_values.append((name, "取得エラー"))
        
        progress.value = i + 1
    
    # KPIカード表示
    clear_output(wait=True)
    
    cards_html = ""
    colors = ['#667eea', '#f093fb', '#feca57', '#48dbfb']
    
    for (name, value), color in zip(kpi_values, colors):
        cards_html += f"""
        <div style="display: inline-block; width: 23%; margin: 1%; 
                    background: linear-gradient(135deg, {color} 0%, {color}dd 100%);
                    color: white; padding: 20px; border-radius: 15px; 
                    box-shadow: 0 4px 6px rgba(0,0,0,0.1); text-align: center;">
            <div style="font-size: 12px; opacity: 0.9;">{name}</div>
            <div style="font-size: 28px; font-weight: bold; margin-top: 10px;">{value}</div>
        </div>
        """
    
    display(HTML(f"""
    <div style="background-color: #f8f9fa; padding: 20px; border-radius: 10px;">
        {cards_html}
    </div>
    <div style="text-align: right; margin-top: 10px; color: #666;">
        🔄 最終更新: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
    </div>
    """))
    
    # 更新ボタン
    refresh_button = widgets.Button(
        description='🔄 KPI更新',
        button_style='primary',
        layout=widgets.Layout(width='150px')
    )
    
    output_kpi = widgets.Output()
    
    def refresh_kpi(b):
        with output_kpi:
            clear_output(wait=True)
            create_kpi_dashboard()
    
    refresh_button.on_click(refresh_kpi)
    display(refresh_button)
    display(output_kpi)

# KPIダッシュボード表示
create_kpi_dashboard()

---

## 💡 使い方ガイド

### よくある質問パターン

#### 📈 売上・収益分析
- `2024年1月の売上合計を教えて`
- `前年同期比での成長率を計算して`
- `売上トップ10の顧客と金額を見せて`

#### 🏢 顧客分析
- `company_sizeがENTの顧客一覧`
- `新規顧客の獲得推移を月別で`
- `業界別の顧客分布を教えて`

#### 🎯 商談分析
- `ステージ別の商談数と金額`
- `平均商談期間を計算して`
- `成約率の高い営業担当者ランキング`

### 💡 分析のコツ

1. **具体的に質問する**
   - ❌ 売上は？
   - ✅ 2024年1月の売上合計は？

2. **期間を明確にする**
   - 今月、先月、2024年Q1など

3. **条件を追加する**
   - 金額1000万円以上
   - ステージがClosed Won
   - company_sizeがENT

---

## 🚀 さらなる活用

- **定期レポート作成**: 分析履歴をCSVエクスポート
- **KPIモニタリング**: KPIダッシュボードを定期確認
- **チーム共有**: ノートブックのURLを共有

---

**サポート**: #data-analysis-hub | data-support@caddi.jp